In [29]:
import pandas as pd
from glob import glob
import csv
import numpy as np

In [30]:
def checkIndexValues(dataSlice):
    """
        Checks if a slice of a pandas data series is simply the row numbers, exported previously
        as an index column.
        
        Parameters
        ----------
        dataSlice: pandas data series
            A slice of data or list
            
        Returns
        --------
        boolean
            True if data values are indices based on row number
            False if data values are not indices based on row number
    """
    for idx, val in enumerate(dataSlice):
        if val != idx+1:
            return False
    return True

In [31]:
# Test
print([1,2,3,4])
print(f"Index values of list: {checkIndexValues([1,2,3,4])}")
print([5,2,1,0])
print(f"Index values of list: {checkIndexValues([5,2,1,0])}")

[1, 2, 3, 4]
Index values of list: True
[5, 2, 1, 0]
Index values of list: False


In [32]:
def checkIfHasLeadingG(value):
    """
        Checks if the first character in a string value is a "G"
        and if the rest of the string is an otherwise valid integer. 
        Used to remove artifact leading "G"s from join columns.
        
        Parameters
        ----------
        value: string
            Value to test
            
        Returns
        --------
        boolean
            True if has a G leading a valid integer 
            False if not
        
    """
    # This only applies to string values
    if not isinstance(value, str):
        return False
    
    # If length is zero, this throws an error
    try:
        if value[0] != "G":
            return False
    except:
        return False
    
    # Try to convert the value, sliced from the first index onward to an int
    # If succeeds, return true, all other conditions return false
    try:
        int(value[1:])
        return True
    except:
        return False

In [33]:
# Test
print('G1234')
print(f"Has leading G of example value: {checkIfHasLeadingG('G1234')}")
print('GX1234')
print(f"Has leading G of example value: {checkIfHasLeadingG('GX1234')}")
print('1234')
print(f"Has leading G of example value: {checkIfHasLeadingG('1234')}")

G1234
Has leading G of example value: True
GX1234
Has leading G of example value: False
1234
Has leading G of example value: False


In [80]:
def inferTypes(file):
    """
        Thoroughly checks for numeric values that should remain as strings due to 
        leading 0s that would otherwise get dropped in pandas import.
        
        Parameters
        ----------
        file: string
            filepath to CSV to open
            
        Returns
        --------
        cols: dictionary
            Relevant columns and string labels for optional pandas import parameter
        
    """
    # Declare outside of scope, since I'm still living in a javascript kinda mind
    cols = {}
    colNames = []

    # Open CSV plainly, no type inference
    with open(file, newline='') as csvfile:
        # Access and loop through rows
        data = csv.reader(csvfile, delimiter=' ', quotechar='|')
        for idx, row in enumerate(data): 
            entries = row[0].split(',')
            # Loop through entries, ignore blank values (index column)
            # Trim extraneous single or double quotes eg: ' " Value " '
            for entry in entries:
                if len(entry) == 0:
                    continue 
                    
                if entry[0] == '"' or entry[0] == '"':
                    entry = entry[1:]
                    
                if entry[-1] == '"' or entry[-1] == '"':
                    entry = entry[:-1]
            # If on first iteration, use values as column names, and iterate
            if idx == 0:
                colNames = entries
                continue
            # Check for string values with leading 0, declare as strings in cols dictionary
            for i, val in enumerate(entries):
                val = str(val)[1:-1]
                if isinstance(val, str) and len(val) > 0 and val[0] == '0':
                    try:
                        int(val)
                        cols[colNames[i]] = 'str'
                    except:
                        continue
    return cols

In [35]:
# Use glob to find all files that match file pattern
files = glob('../data_final/*.csv')
print(files[0:5])

['../data_final/PS05_2017_S.csv', '../data_final/PS06_2019_S.csv', '../data_final/DS01_S.csv', '../data_final/BE03_T.csv', '../data_final/BE03_C.csv']


In [36]:
# Test
print(f'Types for {files[0]}')
print(inferTypes(files[0]))

Types for ../data_final/PS05_2017_S.csv
{'Year': 'str'}


In [37]:
expectedLengths = {
    'C':5,
    'Z':5,
    'T':11,
    'S':2
}

joinCols = {
    'C':'COUNTYFP',
    'Z':'ZCTA',
    'T':'GEOID',
    'S':'STATEFP',
}

expectedColumnLengths = [
    {
        "name":'COUNTYFP',
        "length":5
    },
    {
        "name":'STATEFP',
        "length":2
    },
    {
        "name":'ZCTA',
        "length":5
    },
    {
        "name":'GEOID',
        "length":11
    },
]


def pad(x, length):
    try:
        if len(f"{x}") == length:
            return x
        
        if len(f"{x}") > length:
            return f"{x}"[len(f"{x}")-length:]
        
        if len(f"{x}") != length:
            return f"0{x}"
    except:
        return x
    return x

In [85]:
files = ['../data_final/EC04_T.csv']

In [164]:
# The main loop!
# High level: Open each file, infer types. 
# Find index column, drop it. 
# Find leading G's, remove them.

# Start by looping through each fie
for file in files:
    
    # Infer types in order to figure out which numeric fields need to remain strings
    types = inferTypes(file)
    # Use those types and read in to pandas
    raw = pd.read_csv(file, encoding='latin-1', dtype=types)
    
    # Find the join column based on the last character before the file extension
    try:
        joinCol = joinCols[file.split('.csv')[0][-1]]
    # Others have it in the middle - (ツ)_/¯
    except:
        joinCol = joinCols[file.split('_')[2]]
    
    # IF the join column is not as expected, something is wrong. Some have a vestigial GISJOIN column
    # Otherwise, flag this to check out after the back
    if joinCol not in list(raw.columns):
        print(f'Warning - {file} does not have the proper join column')
        if 'GISJOIN' in list(raw.columns):
            pass
        else:
            continue
            
    for column in raw.columns:
        # If the dreaded GISJOIN, change the column name to the appropriate geographic join
        if column == 'GISJOIN':
            raw = raw.drop(columns=[joinCol])
            tempColumns = list(raw.columns)
            tempColumns[list(raw.columns).index('GISJOIN')] = joinCol
            raw.columns = tempColumns
            column = joinCol
            
        # Markers of an unnamed index column to be removed
        if (column == 'Unnamed: 0' or column == 'X') and checkIndexValues(raw[column][0:10]):
            raw = raw.drop(columns=[column])
            continue
            
        # Find leading G
        if checkIfHasLeadingG(raw[column][0]):
            raw.loc[:,column] = raw.loc[:,column].str.slice(1,-1)
    
    # This section is meant to appropriately pad with leading zeroes
    # However, some columns are not formatted consistently, such as just a county code, not state+county for countyFP
    # Better solution needed.
#     for col in expectedColumnLengths:
#         if col['name'] in list(raw.columns):
#             raw.loc[:, col['name']] = raw.loc[:, col['name']].apply(lambda x: pad(x, col['length']))
    
#     # Check for un-concatenated county and state fp columns
#     if 'STATEFP' in list(raw.columns) and 'COUNTYFP' in list(raw.columns):
#         if len(raw.iloc[0].COUNTYFP) == 3:
#             raw['COUNTYFP'] = raw['STATEFP'].astype(str) + raw['COUNTYFP']
            
#     # Correct missing or null STATEFP based on geoid or countyfp
#     if 'STATEFP' in list(raw.columns):
#         if 'GEOID' in list(raw.columns):
#             raw['STATEFP'] = raw['GEOID'].astype(str).str.slice(0,2)
#         elif 'COUNTYFP' in list(raw.columns):
#             raw['STATEFP'] = raw['COUNTYFP'].astype(str).str.slice(0,2)
            
#     # Correct missing or null STATEFP based on geoid or countyfp
#     if 'COUNTYFP' in list(raw.columns):
#         if 'GEOID' in list(raw.columns):
#             raw['COUNTYFP'] = raw['GEOID'].astype(str).str.slice(0,5)
            
    raw.round(2).to_csv(file, index=False)

In [145]:
rw

,name1,name2,street1,street2,city,state,zip,zip4,category,countyGEOID,countyName,source,geom,Longitude,Latitude
1,NaN,1601 West 4th Street,NaN,Coffeyville,KS,67337,NaN,buprenorphine,20125.0,Montgomery,SAMHSA,c(-95.64145,37.0402928),-95.641450,37.040293
2,NaN,3751 West Main Street,NaN,Independence,KS,67301,NaN,buprenorphine,20125.0,Montgomery,SAMHSA,c(-95.7566357,37.2256576),-95.756636,37.225658
3,Bartlesville,4100 SE Adams Road,Suite E-108,Bartlesville,OK,74006,NaN,buprenorphine,40147.0,Washington,SAMHSA,c(-95.9301563,36.7455475),-95.930156,36.745548
4,Bartlesville Rightway Medical,610 West Hensley Boulevard,NaN,Bartlesville,OK,74003,NaN,buprenorphine,40147.0,Washington,SAMHSA,c(-95.9840426,36.7535489),-95.984043,36.753549
5,NaN,410 West Main Street,NaN,Barnsdall,OK,74002,NaN,buprenorphine,40113.0,Osage,SAMHSA,c(-96.1618624,36.5618247),-96.161862,36.561825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42131,NaN,4120 Meridian St,NaN,Bellingham,WA,98226,NaN,naltrexone/vivitrol,53073.0,Whatcom,vivitrolWeb,c(-122.485187967534,48.7909350026357),-122.485188,48.790935
42141,NaN,1815 MAIN ST,NaN,Ferndale,WA,98248,NaN,naltrexone/vivitrol,53073.0,Whatcom,vivitrolWeb,c(-122.580530020679,48.8433449896254),-122.580530,48.843345
42151,NaN,8071 GUIDE MERIDIAN ROAD,NaN,Lynden,WA,98264,NaN,naltrexone/vivitrol,53073.0,Whatcom,vivitrolWeb,c(-122.487691977288,48.9342380123838),-122.487692,48.934238
42161,NaN,401 2nd Ave.,NaN,Rolette,ND,58366,NaN,naltrexone/vivitrol,38079.0,Rolette,vivitrolWeb,c(-99.8463525946865,48.6620162483217),-99.846353,48.662016


In [147]:
rw = pd.read_csv('../data_final/EC04.csv')

In [156]:
rw['COUNTYFP'] = rw['GEOID'].astype(str).str.slice(0,2) + rw['GEOID'].astype(str).str.slice(3,)

In [157]:
rw['COUNTYFP'] = rw['COUNTYFP'].apply(lambda x: pad(x, 5))

In [160]:
rw = rw.drop(columns=["GEOID"])

In [163]:
rw.to_csv('../data_final/DS04_C.csv', index=False)